In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Thiết lập hiển thị
sns.set_theme(style="whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Hiển thị số thập phân gọn gàng

# 2. Định nghĩa đường dẫn (Lùi ra 1 cấp từ src/)
BASE_DIR = '..'
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', 'paris')
FIGURES_DIR = os.path.join(BASE_DIR, 'figures')

# Tạo thư mục chứa hình nếu chưa có
os.makedirs(FIGURES_DIR, exist_ok=True)

print("--- BẮT ĐẦU MỤC 4: TỔNG HỢP KPI ---")

# 3. NẠP DỮ LIỆU ĐÃ LÀM SẠCH
# Tìm tất cả file listings_processed.csv trong thư mục brussels
all_files = glob.glob(os.path.join(PROCESSED_DIR, '*', 'listings_processed.csv'))

df_list = []
print(f"Đang nạp dữ liệu từ {len(all_files)} snapshots...")

for filename in all_files:
    # Lấy tên snapshot từ tên thư mục cha (ví dụ: '21 June, 2025')
    snapshot_name = os.path.basename(os.path.dirname(filename))
    
    # Đọc file (low_memory=False để tránh cảnh báo nếu file lớn)
    df = pd.read_csv(filename, low_memory=False)
    
    # Thêm cột thời gian để phân biệt dữ liệu của tháng nào
    df['snapshot_date'] = snapshot_name
    df_list.append(df)

# Gộp thành 1 DataFrame lớn
df_all = pd.concat(df_list, ignore_index=True)

# Chuyển đổi cột thời gian sang datetime để sắp xếp đúng thứ tự (Quá khứ -> Tương lai)
df_all['snapshot_datetime'] = pd.to_datetime(df_all['snapshot_date'])
df_all = df_all.sort_values('snapshot_datetime')

print(f"✅ Đã nạp thành công {len(df_all)} dòng dữ liệu.")
print("Các snapshot có trong dữ liệu:")
print(df_all['snapshot_date'].value_counts())

--- BẮT ĐẦU MỤC 4: TỔNG HỢP KPI ---
Đang nạp dữ liệu từ 4 snapshots...
✅ Đã nạp thành công 343003 dòng dữ liệu.
Các snapshot có trong dữ liệu:
snapshot_date
06 December, 2024     91031
03 March, 2025        86064
06 June, 2025         84055
12 September, 2025    81853
Name: count, dtype: int64


In [2]:
print("--- ĐANG TÍNH TOÁN KPI ---")
CITY_SUFFIX = 'paris' # Dùng để đặt tên file không bị trùng với Berlin/Paris

# === 1. KPI Nguồn cung (Total Listings) ===
# Đếm số lượng listing ID theo từng snapshot
kpi_supply = df_all.groupby('snapshot_date')['id'].count().reset_index(name='total_listings')

# === 2. KPI Giá (Median Price) ===
# Lọc dữ liệu sạch để tính giá:
# - Điều kiện 1: Không bị cờ giá 0 (qa_flag_price_zero == False)
# - Điều kiện 2: Giá không bị rỗng/NaN (notna) - (Vì ta có ~1000 dòng bị NaN)
df_valid_price = df_all[
    (df_all['qa_flag_price_zero'] == False) & 
    (df_all['price_numeric'].notna())
]
# Tính trung vị (Median) thay vì trung bình (Mean) để tránh bị nhiễu bởi các căn hộ siêu đắt
kpi_price = df_valid_price.groupby('snapshot_date')['price_numeric'].median().reset_index(name='median_price')

# === 3. KPI Cơ cấu Loại phòng (Room Type Mix) ===
# Bước 1: Đếm số lượng từng loại phòng
kpi_room = df_all.groupby(['snapshot_date', 'room_type']).size().reset_index(name='count')
# Bước 2: Tính tổng số listing của mỗi snapshot
total_by_snapshot = df_all.groupby('snapshot_date')['id'].count().reset_index(name='total')
# Bước 3: Gộp lại để tính phần trăm
kpi_room = kpi_room.merge(total_by_snapshot, on='snapshot_date')
kpi_room['percentage'] = (kpi_room['count'] / kpi_room['total']) * 100

# === HIỂN THỊ KẾT QUẢ (Kiểm tra nhanh) ===
print("\n>> Bảng KPI Nguồn Cung:")
print(kpi_supply)

print("\n>> Bảng KPI Giá Trung vị:")
print(kpi_price)

print("\n>> Bảng KPI Loại phòng (5 dòng đầu):")
print(kpi_room[['snapshot_date', 'room_type', 'percentage']].head())

# === LƯU FILE (YÊU CẦU CỦA ĐỀ BÀI) ===
# Lưu vào thư mục processed/paris/ với tên có đuôi _brussels
kpi_supply.to_csv(os.path.join(PROCESSED_DIR, f'kpi_supply_{CITY_SUFFIX}.csv'), index=False)
kpi_price.to_csv(os.path.join(PROCESSED_DIR, f'kpi_price_{CITY_SUFFIX}.csv'), index=False)
kpi_room.to_csv(os.path.join(PROCESSED_DIR, f'kpi_room_type_{CITY_SUFFIX}.csv'), index=False)

print(f"\n✅ Đã hoàn tất Mục 4. Các file KPI đã được lưu tại: {PROCESSED_DIR}")

--- ĐANG TÍNH TOÁN KPI ---

>> Bảng KPI Nguồn Cung:
        snapshot_date  total_listings
0      03 March, 2025           86064
1   06 December, 2024           91031
2       06 June, 2025           84055
3  12 September, 2025           81853

>> Bảng KPI Giá Trung vị:
       snapshot_date  median_price
0     03 March, 2025        146.00
1  06 December, 2024        150.00
2      06 June, 2025        161.00

>> Bảng KPI Loại phòng (5 dòng đầu):
       snapshot_date        room_type  percentage
0     03 March, 2025  Entire home/apt       88.84
1     03 March, 2025       Hotel room        0.79
2     03 March, 2025     Private room       10.19
3     03 March, 2025      Shared room        0.17
4  06 December, 2024  Entire home/apt       89.06

✅ Đã hoàn tất Mục 4. Các file KPI đã được lưu tại: ..\processed\paris
